### Using the numba cuda simulator for debugging

Execution of kernels is performed by the simulator one block at a time. One thread is spawned for each thread in the block, and scheduling of the execution of these threads is left up to the operating system.

*for me it just did the thread sequentially

In [2]:
import os
os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

In [3]:
from numba import cuda
import numpy as np

In [4]:
@cuda.jit
def upsweep_kernel(input:np.array ,two_d:int, n:int):
    two_dplus1 = 2*two_d
    #debug: need to do one operation in a stride of two_dplus1 and the data doesn't touch
    #debug: have the last thread of the stride do the work

    bx, tx = cuda.blockIdx.x, cuda.threadIdx.x
    idx = cuda.blockDim.x*bx + tx

    if idx%two_dplus1==0 and idx+two_dplus1-1 < n:
        input[idx+two_dplus1-1] += input[idx+two_d-1]

@cuda.jit 
def downsweep_kernel(input:np.array ,two_d:int, n:int):
    two_dplus1 = 2*two_d

    bx, tx = cuda.blockIdx.x, cuda.threadIdx.x
    idx = cuda.blockDim.x*bx + tx

    if idx%two_dplus1==0 and idx+two_dplus1-1 < n:
        t = input[int(idx+two_d-1)]
        input[int(idx+two_d-1)] = input[int(idx+two_dplus1-1)]
        input[int(idx+two_dplus1-1)] += t

In [5]:
def predix_sum(input:np.array):
    x = input.copy()
    n = input.shape[0]
    two_d = 1
    tpb = 512
    nb = (n + tpb - 1)//tpb
    while two_d<=n//2:
        upsweep_kernel[nb,tpb](x,two_d,n)
        print(x)
        two_d*=2
    x[n-1]=0
    print(x)
    two_d=n/2
    while two_d>=1:
        downsweep_kernel[nb,tpb](x,two_d,n) 
        print(x)
        two_d/=2

   

In [6]:

input  = [1,2,3,4,5,6,7,8]
output = [0]*8
input = np.array(input)
output = np.array(output)


In [7]:
predix_sum(input)

[ 1  3  3  7  5 11  7 15]
[ 1  3  3 10  5 11  7 26]
[ 1  3  3 10  5 11  7 36]
[ 1  3  3 10  5 11  7  0]
[ 1  3  3  0  5 11  7 10]
[ 1  0  3  3  5 10  7 21]
[ 0  1  3  6 10 15 21 28]


[1 3 3 10 5 11 7 36 0]

In [8]:
# came to my notice that you just need 1 thread per block
@cuda.jit
def upsweep_kernel(input:np.array ,two_d:int, n:int):
    two_dplus1 = 2*two_d
    #debug: need to do one operation in a stride of two_dplus1 and the data doesn't touch
    #debug: have the last thread of the stride do the work

    bx, tx = cuda.blockIdx.x, cuda.threadIdx.x
    idx = (cuda.blockDim.x*bx + tx)*two_dplus1

    if idx%two_dplus1==0 and idx+two_dplus1-1 < n:
        input[idx+two_dplus1-1] += input[idx+two_d-1]

@cuda.jit 
def downsweep_kernel(input:np.array ,two_d:int, n:int):
    two_dplus1 = 2*two_d

    bx, tx = cuda.blockIdx.x, cuda.threadIdx.x
    idx = (cuda.blockDim.x*bx + tx)*two_dplus1

    if idx%two_dplus1==0 and idx+two_dplus1-1 < n:
        t = input[int(idx+two_d-1)]
        input[int(idx+two_d-1)] = input[int(idx+two_dplus1-1)]
        input[int(idx+two_dplus1-1)] += t

In [9]:
def predix_sum(input:np.array):
    x = input.copy()
    n = input.shape[0]
    two_d = 1
    tpb = 512
    
    while two_d<=n//2:
        nb = (n//(two_d*2) + tpb - 1)//tpb
        upsweep_kernel[nb,tpb](x,two_d,n)
        print(x)
        two_d*=2
    x[n-1]=0
    print(x)
    two_d=n/2
    while two_d>=1:
        nb = (n//(two_d*2) + tpb - 1)//tpb
        print(n//(two_d*2))
        downsweep_kernel[int(nb),tpb](x,two_d,n) 
        print(x)
        two_d/=2
    output = np.cumsum(input)-input

    for i in range(n):
        if x[i]!=output[i]:
            print("error at index",i)

In [13]:
# np.set_printoptions(threshold=32)

#make test for 4096 elements
input  = [i for i in range(int(8388608/16))]
output = [0]*16777216
input = np.array(input)
output = np.array(output)

predix_sum(input)


[      0       1       2 ... 1048569  524286 1048573]
[      0       1       2 ... 1048569  524286 2097142]
[      0       1       2 ... 1048569  524286 4194268]
[      0       1       2 ... 1048569  524286 8388472]
[       0        1        2 ...  1048569   524286 16776688]
[       0        1        2 ...  1048569   524286 33552352]
[       0        1        2 ...  1048569   524286 67100608]
[        0         1         2 ...   1048569    524286 134184832]
[        0         1         2 ...   1048569    524286 268304128]
[        0         1         2 ...   1048569    524286 536346112]
[         0          1          2 ...    1048569     524286 1071643648]
[         0          1          2 ...    1048569     524286 2139092992]
[         0          1          2 ...    1048569     524286 4261408768]
[         0          1          2 ...    1048569     524286 8455708672]
[          0           1           2 ...     1048569      524286
 16642981888]
[          0           1           2 ..